In [ ]:
# data processing and creating a map with location, values key-value pair
import os
import pandas as pd 
import numpy as np

directory = "1. Target Variables" 

# goes over directory and appends all the read csvs into DataFrames
arr_dataframes = []
for filename in os.listdir(directory):
    path= os.path.join(directory, filename)
    data = pd.read_csv(path)
    arr_dataframes.append(data)

loc_map = {}

# first for loop to iterate over and create array
# second for loop to append to same index locations
for row in arr_dataframes[0].values:
    location = row[2]
    value = row[8]
    if location not in loc_map: # gets column 2 which is Region
        loc_map[location] = [[value]] #inputs in first value which is 
    else:
        loc_map[location].append([value])
        
# go over each csv read from each file and set key to location like 'Alberta' and have an array of values 
# should be size 2 for each array 
for csv in arr_dataframes[1:]:
    arr = csv.values
    index = 0 # reset count for each csv and each location
    firstLocation = 'Alberta'
    for row in arr:
        location = row[2]
        value = row[8]
        if firstLocation != location:
            firstLocation = location
            index = 0
        try:
            loc_map[location][index].append(value)
        except:
            print(location)
            print(value)
            print(index)
        
        index += 1
            

In [ ]:
# creating X_train, Y_train, X_test, Y_test datasets for input to ML model out of region
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy

# cell will be used to make location map into np array that can be called to train

    
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        dataX.append(dataset[i:i+look_back])
        dataY.append([dataset[i + look_back]])
    return numpy.array(dataX), numpy.array(dataY)

# split into train and test sets

dataset = loc_map['Manitoba'] # change here for region in the mapping to get the arrays there
sc = MinMaxScaler(feature_range = (0, 1))
dataset = sc.fit_transform(dataset)

#train_size = int(len(dataset) * 1)
#test_size = len(dataset) - train_size
#train, test = dataset[0:train_size], dataset[train_size:len(dataset)]
train = dataset[330:]

# reshape into X=t and Y=t+1
look_back = 12
trainX, trainY = create_dataset(train, look_back)

trainY = trainY.squeeze()
#testY = testY.squeeze()
batch_size = trainX.shape[0]
timestamps = 12
features = 2


In [ ]:
# training model 
from keras import losses
from keras.layers import Dense, Activation, Dropout
model = Sequential()
model.add(LSTM(32, input_shape=(timestamps, features), return_sequences=True))
model.add(LSTM(32, return_sequences=False))
#model.add(LSTM(4, batch_input_shape=(batch_size, timestamps, 1), stateful=True))
model.add(Dense(10))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(2)) # output 1 in end 
model.compile(loss=losses.mean_squared_error, optimizer='adam')
model.fit(trainX[:], trainY[:], epochs=100, batch_size=32, verbose=2)

In [ ]:
# INPUT IN LAST YEAR OF DATA AS MODEL PREDICTS NEXT YEARS WORTH
# will create a 2D array with [deliveries, price] as each array 
# for quebec
arr1 = [
    2397,
1215,
1749,
869,
1124,
822,
168,
220,
4823,
5934,
3950,
1508

]
arr2 = [
    497.65,
500.71,
497.06,
500.86,
507.79,
516.13,
510.6,
490,
493,
493,
498,
503

]
arr = []
for i in range(12):
    arr.append([arr1[i], arr2[i]])
print(arr)


In [ ]:
# predict model on the data from last cell, it will print next years worth
X = np.array(arr)
sc = MinMaxScaler(feature_range = (0, 1))
X = sc.fit_transform(X)

print(X.shape)

for i in range(12):
    nextValue = model.predict(X.reshape(1, X.shape[0], X.shape[1]))
    readable = print(sc.inverse_transform(nextValue))
    print(nextValue) # then move on
    
    array = [] 
    for i in range(1, len(X)):
        array.append(X[i].tolist())
    array.append(nextValue.tolist()[0])
    
    array = np.array(array)
    X = array
    


In [ ]:
# for sasquatch
arr1 = [961341,
786206,
828735,
757098,
733139,
894089,
686580,
643389,
1210007,
1245927,
1057832,
809909



]
arr2 = [
 498.8,
505.44,
503.52,
507.93,
526.42,
523.05,
526.38,
506.94,
479.04,
480.26,
476.7,
475.42



]

arr = []
for i in range(12):
    arr.append([arr1[i], arr2[i]])
print(arr)

In [ ]:

# for quebec
arr1 = [
    2397,
1215,
1749,
869,
1124,
822,
168,
220,
4823,
5934,
3950,
1508

]
arr2 = [
    497.65,
500.71,
497.06,
500.86,
507.79,
516.13,
510.6,
490,
493,
493,
498,
503

]
arr = []
for i in range(12):
    arr.append([arr1[i], arr2[i]])
print(arr)

In [ ]:
# for ontario - improved score on last 12 months
arr1 = [
    713,
1781,
2920,
910,
922,
1638,
1650,
2377,
16472,
7557,
1461,
2049

]
arr2 = [
    512,
505.02,
527.91,
531.26,
531.91,
516.17,
547.35,
510.48,
495.46,
489,
488,
466.01

]
arr = []
for i in range(12):
    arr.append([arr1[i], arr2[i]])
print(arr)

In [ ]:
# for Manitoba - going to see score change
arr1 = [
    179708,
167022,
183448,
182708,
209767,
240994,
229022,
291794,
375382,
211474,
220899,
183603

]
arr2 = [
    497.6,
505.69,
508.17,
517.98,
527.46,
526.19,
521.03,
497.72,
473.98,
477.19,
476.67,
481.08

]
arr = []
for i in range(12):
    arr.append([arr1[i], arr2[i]])
print(arr)

In [ ]:
# for alberta
arr1 = [
    513298,
318556,
629320,
483659,
548214,
461929,
250989,
174771,
182911,
707299,
557669,
375388

]
arr2 = [
    493.8,
490.42,
496.66,
507.28,
516.82,
520.12,
516.65,
508.03,
484.82,
477.07,
472.02,
473.39

]
arr = []
for i in range(12):
    arr.append([arr1[i], arr2[i]])
print(arr)

In [ ]:
# for british columbia
arr1 = [
    14200,
4100,
3400,
5900,
3100,
0,
0,
700,
12100,
23900,
22400,
8300,

]
arr2= [
    493.8,
490.42,
496.66,
507.28,
516.82,
520.12,
516.65,
508.03,
484.82,
477.07,
472.02,
473.39

]
arr = []
for i in range(12):
    arr.append([arr1[i], arr2[i]])
print(arr)